In [1]:
import pandas as pd

In [4]:
str_cols = ["SCHEDULED_DEPARTURE", "DEPARTURE_TIME","WHEELS_OFF","WHEELS_ON","SCHEDULED_ARRIVAL","ARRIVAL_TIME"]
df_flights=pd.read_csv("data/flights.csv",dtype={i:str for i in str_cols})
df_airlines=pd.read_csv("data/airlines.csv")
df_airports=pd.read_csv("data/airports.csv")

# Find top 5 most delayed airline (arrival)

airline code, airline name, num of flights, num origin, num destin, num non cancelled flight, delay ratio (exclude cancellation)

In [7]:
df_flights.head(5).T

,0,1,2,3,4
YEAR,2015,2015,2015,2015,2015
MONTH,1,1,1,1,1
DAY,1,1,1,1,1
DAY_OF_WEEK,4,4,4,4,4
AIRLINE,AS,AA,US,AA,AS
FLIGHT_NUMBER,98,2336,840,258,135
TAIL_NUMBER,N407AS,N3KUAA,N171US,N3HYAA,N527AS
ORIGIN_AIRPORT,ANC,LAX,SFO,LAX,SEA
DESTINATION_AIRPORT,SEA,PBI,CLT,MIA,ANC
SCHEDULED_DEPARTURE,0005,0010,0020,0020,0025


### df_flights: airline code, flight number (count of FLIGHT_NUMBER), orign_airport (count), cancelled flights (sum of CANCELLED)

In [24]:
df_result=df_flights.groupby(by="AIRLINE").agg({"FLIGHT_NUMBER":"count","CANCELLED":"sum","ORIGIN_AIRPORT":"nunique"})

df_result

In [27]:
df_result["FLIGHT_OPERATED"]=df_result["FLIGHT_NUMBER"]-df_result["CANCELLED"]

In [28]:
df_result

,FLIGHT_NUMBER,CANCELLED,ORIGIN_AIRPORT,FLIGHT_OPERATED
AIRLINE,,,,
AA,725984,10919,276,715065
AS,172521,669,190,171852
B6,267048,4276,184,262772
DL,875881,3824,440,872057
EV,571977,15231,491,556746
F9,90836,588,155,90248
HA,76272,171,51,76101
MQ,294632,15025,351,279607
NK,117379,2004,98,115375


### group by airline, find num delayed flights

In [43]:
df_flights["ISDELAYED"]=df_flights["ARRIVAL_DELAY"]>0

In [44]:
df_flights["ISDELAYED"].astype(int)

0          0
1          0
2          1
3          0
4          0
          ..
5819074    0
5819075    0
5819076    0
5819077    0
5819078    1
Name: ISDELAYED, Length: 5819079, dtype: int32

In [76]:
df_result2=df_flights.groupby("AIRLINE").agg({"ISDELAYED":"sum"})

In [77]:
df_result2

,ISDELAYED
AIRLINE,
AA,252191.0
AS,56953.0
B6,101998.0
DL,250840.0
EV,213217.0
F9,41232.0
HA,30179.0
MQ,103505.0
NK,56887.0


In [78]:
df_result_final=pd.merge(df_result,df_result2,how="left",on="AIRLINE")

In [79]:
df_result_final

,FLIGHT_NUMBER,CANCELLED,ORIGIN_AIRPORT,FLIGHT_OPERATED,ISDELAYED
AIRLINE,,,,,
AA,725984,10919,276,715065,252191.0
AS,172521,669,190,171852,56953.0
B6,267048,4276,184,262772,101998.0
DL,875881,3824,440,872057,250840.0
EV,571977,15231,491,556746,213217.0
F9,90836,588,155,90248,41232.0
HA,76272,171,51,76101,30179.0
MQ,294632,15025,351,279607,103505.0
NK,117379,2004,98,115375,56887.0


In [83]:
df_result_final["DELAY_RATIO"]=df_result_final["ISDELAYED"]/df_result_final["FLIGHT_OPERATED"]

In [84]:
df_result_final["DELAY_RATIO"]=df_result_final["DELAY_RATIO"].round(3)

In [90]:
df_result_final

,FLIGHT_NUMBER,CANCELLED,ORIGIN_AIRPORT,FLIGHT_OPERATED,ISDELAYED,DELAY_RATIO
AIRLINE,,,,,,
NK,117379,2004,98,115375,56887.0,0.493
F9,90836,588,155,90248,41232.0,0.457
HA,76272,171,51,76101,30179.0,0.397
VX,61903,534,57,61369,24180.0,0.394
US,198715,4067,79,194648,76285.0,0.392
B6,267048,4276,184,262772,101998.0,0.388
OO,588353,9960,546,578393,222435.0,0.385
EV,571977,15231,491,556746,213217.0,0.383
WN,1261855,16043,258,1245812,470767.0,0.378


In [106]:
df_result_final=df_result_final.sort_values(by=["DELAY_RATIO"],ascending=False)

In [107]:
df_result_final

,FLIGHT_NUMBER,CANCELLED,ORIGIN_AIRPORT,FLIGHT_OPERATED,ISDELAYED,DELAY_RATIO
AIRLINE,,,,,,
NK,117379,2004,98,115375,56887.0,0.493
F9,90836,588,155,90248,41232.0,0.457
HA,76272,171,51,76101,30179.0,0.397
VX,61903,534,57,61369,24180.0,0.394
US,198715,4067,79,194648,76285.0,0.392
B6,267048,4276,184,262772,101998.0,0.388
OO,588353,9960,546,578393,222435.0,0.385
EV,571977,15231,491,556746,213217.0,0.383
WN,1261855,16043,258,1245812,470767.0,0.378


In [110]:
df_result3=pd.merge(df_result_final,df_airlines,how="left",left_on="AIRLINE",right_on="IATA_CODE")

In [111]:
df_result3

,FLIGHT_NUMBER,CANCELLED,ORIGIN_AIRPORT,FLIGHT_OPERATED,ISDELAYED,DELAY_RATIO,IATA_CODE,AIRLINE
0,117379,2004,98,115375,56887.0,0.493,NK,Spirit Air Lines
1,90836,588,155,90248,41232.0,0.457,F9,Frontier Airlines Inc.
2,76272,171,51,76101,30179.0,0.397,HA,Hawaiian Airlines Inc.
3,61903,534,57,61369,24180.0,0.394,VX,Virgin America
4,198715,4067,79,194648,76285.0,0.392,US,US Airways Inc.
5,267048,4276,184,262772,101998.0,0.388,B6,JetBlue Airways
6,588353,9960,546,578393,222435.0,0.385,OO,Skywest Airlines Inc.
7,571977,15231,491,556746,213217.0,0.383,EV,Atlantic Southeast Airlines
8,1261855,16043,258,1245812,470767.0,0.378,WN,Southwest Airlines Co.
9,294632,15025,351,279607,103505.0,0.370,MQ,American Eagle Airlines Inc.


In [114]:
df_result4=df_result3[["IATA_CODE","AIRLINE","ORIGIN_AIRPORT","FLIGHT_NUMBER","CANCELLED","FLIGHT_OPERATED","ISDELAYED","DELAY_RATIO"]]

In [117]:
df_result4.sort_values(by="DELAY_RATIO",ascending=False).head(5)

,IATA_CODE,AIRLINE,ORIGIN_AIRPORT,FLIGHT_NUMBER,CANCELLED,FLIGHT_OPERATED,ISDELAYED,DELAY_RATIO
0,NK,Spirit Air Lines,98,117379,2004,115375,56887.0,0.493
1,F9,Frontier Airlines Inc.,155,90836,588,90248,41232.0,0.457
2,HA,Hawaiian Airlines Inc.,51,76272,171,76101,30179.0,0.397
3,VX,Virgin America,57,61903,534,61369,24180.0,0.394
4,US,US Airways Inc.,79,198715,4067,194648,76285.0,0.392
